# 📚 Kapitola 18: REST API vytvoření s FastAPI

<div style="background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h2 style="color: white; margin: 0;">Blok 2 | INTERMEDIATE</h2>
    <p style="color: white; margin: 10px 0;">📖 Výuková kapitola</p>
</div>

## 🎯 Co se naučíte

V této kapitole se zaměříme na následující témata:

- **FastAPI installation a setup**
- **Path a query parameters**
- **Request/Response models (Pydantic)**
- **Data validation**
- **Middleware a dependencies**
- **Authentication a authorization**
- **Automatic API documentation**

## ⚠️ Předpoklady
Tato kapitola navazuje na kapitoly: 13, 16

---


In [ ]:
# ⚙️ Inicializace prostředí
# Tento kód nastavuje prostředí pro kapitolu 18

import sys
import os
from datetime import datetime

# Informace o prostředí
print("=" * 60)
print(f"📚 KAPITOLA 18: REST API vytvoření s FastAPI")
print("=" * 60)
print(f"🐍 Python verze: {sys.version}")
print(f"📅 Datum spuštění: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💻 OS: {os.name}")
print(f"📁 Pracovní adresář: {os.getcwd()}")
print("=" * 60)

# Instalace potřebných knihoven (odkomentujte podle potřeby)
# !pip install requests pandas numpy matplotlib
# !pip install beautifulsoup4 sqlalchemy fastapi

# Import základních knihoven
import json
import csv
import time
import random
from pathlib import Path
from typing import List, Dict, Optional, Any

print("✅ Prostředí připraveno!")

## 📖 Teoretická část

<div style="background: #f0f4ff; padding: 20px; border-left: 5px solid #4a69bd; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🎓 Základní teorie a koncepty</h3>
</div>

# REST API vytvoření s FastAPI

## 1. Úvod a motivace

REST API je nejčastější způsob, jak poskytovat datové služby mezi aplikacemi. V dnešní době je tato technologie klíčová pro vývoj webových aplikací, mikroslužeb a integrací systémů. FastAPI je moderní, rychlý a jednoduchý framework pro tvorbu REST API v Pythonu, který se stává populárním volbou pro vývoj produktivních a bezpečných API.

Použití FastAPI najdeš například v backendových aplikacích pro webové stránky, mobilní aplikace, datové služby nebo systémy pro integraci. V praxi se FastAPI používá v oblastech jako jsou e-commerce, bankovnictví, sociální sítě a řídicí systémy.

Po této kapitole budeš umět:
- Nainstalovat a nastavit FastAPI projekt
- Vytvářet REST endpointy s různými parametry
- Validovat data pomocí Pydantic modelů
- Implementovat autentizaci a autorizaci
- Vytvářet automatickou dokumentaci API

## 2. Hlavní koncepty

### FastAPI installation a setup

Začněme instalací FastAPI a jeho závislostí:

```bash
pip install fastapi uvicorn
```

`uvicorn` je ASGI server, který bude spouštět náš API. Vytvořme nejprve základní soubor `main.py`:

```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def root():
    return {"message": "Hello World"}
```

Spusť aplikaci příkazem:

```bash
uvicorn main:app --reload
```

Přepínač `--reload` automaticky restartuje server při změně kódu.

### Path a query parameters

FastAPI umožňuje snadno pracovat s cestami a parametry. Například:

```python
from fastapi import FastAPI, Path

app = FastAPI()

@app.get("/users/{user_id}")
async def read_user(user_id: int = Path(..., ge=1)):
    return {"user_id": user_id}

@app.get("/items/")
async def read_items(q: str = None, skip: int = 0, limit: int = 100):
    return {"q": q, "skip": skip, "limit": limit}
```

V tomto příkladu:
- `user_id` je path parameter
- `q`, `skip`, `limit` jsou query parametry
- Parametr `...` znamená, že je povinný
- `ge=1` nastavuje minimální hodnotu

### Request/Response models (Pydantic)

FastAPI využívá Pydantic pro validaci a serializaci dat. Vytvořme modely:

```python
from pydantic import BaseModel
from typing import Optional

class User(BaseModel):
    id: int
    name: str
    email: str
    is_active: bool = True

class UserCreate(BaseModel):
    name: str
    email: str
```

Použití modelů v API:

```python
@app.post("/users/", response_model=User)
async def create_user(user: UserCreate):
    # Vytvoření uživatele
    return user
```

### Data validation

FastAPI automaticky validuje vstupní data podle Pydantic modelů. Příklad:

```python
from pydantic import BaseModel, EmailStr, validator

class UserCreate(BaseModel):
    name: str
    email: EmailStr
    
    @validator('name')
    def name_must_not_be_empty(cls, v):
        if not v.strip():
            raise ValueError('Name cannot be empty')
        return v
```

### Middleware a dependencies

Middleware umožňují provádět akce před/za každým požadavkem:

```python
from fastapi import Depends, FastAPI, HTTPException

async def get_db():
    # Připojení k databázi
    pass

@app.get("/items/")
async def read_items(db = Depends(get_db)):
    return {"message": "Items"}
```

Dependencies se používají pro sdílení logiky mezi endpointy.

### Authentication a authorization

Jednoduchý příklad autentizace:

```python
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials

security = HTTPBearer()

@app.get("/protected")
async def protected_route(credentials: HTTPAuthorizationCredentials = Depends(security)):
    # Kontrola tokenu
    return {"message": "Access granted"}
```

### Automatic API documentation

FastAPI automaticky generuje dokumentaci přes Swagger UI a ReDoc:

- Swagger UI: `http://localhost:8000/docs`
- ReDoc: `http://localhost:8000/redoc`

## 3. Důležité detaily

Časté chyby zahrnují nevalidní modely, nesprávné použití typů a zapomínání na async/await. Při vývoji API je důležité kontrolovat typovou kompatibilitu a validaci dat.

Best practices zahrnují:
- Vždy používat Pydantic modely pro vstupy i výstupy
- Implementovat správné HTTP status kódy
- Používat Dependency Injection pro sdílení logiky
- Omezovat počet paralelních požadavků na API

Pro lepší výkon:
- Implementovat caching
- Optimalizovat databázové dotazy
- Použít uvicorn s více workers
- Implementovat kompresi dat

## 4. Propojení s předchozími kapitolami

Tato kapitola navazuje na znalosti Pythonu, HTTP protokolu a základních konceptů API. Vývoj REST API rozšiřuje předchozí poznatky o tvorbě aplikací a využívá již známé principy jako asynchronní programování a práce s daty.

FastAPI se tedy stává přirozeným krokem ve vývoji backendu, který zajišťuje bezpečnost, rychlost a snadnou dokumentaci.


## 💻 Praktické příklady

<div style="background: #e8f5e9; padding: 20px; border-left: 5px solid #4caf50; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">👨‍💻 Hands-on příklady ke spuštění</h3>
    <p>Následující příklady si můžete hned vyzkoušet. Každý příklad je samostatně spustitelný.</p>
</div>


# 1. Instalace a nastavení FastAPI
## Co demonstruje:
Základní instalace FastAPI a spuštění jednoduchého serveru.

```python
# app.py
from fastapi import FastAPI

# Vytvoření instance FastAPI aplikace
app = FastAPI(title="Moje první API", version="1.0.0")

# Definice endpointu pro hlavní stránku
@app.get("/")
async def root():
    return {"message": "Vítejte v mé API!"}

# Spuštění serveru příkazem: uvicorn app:app --reload
```

**Očekávaný výstup:**
- Server běží na http://127.0.0.1:8000/
- Výstup: {"message": "Vítejte v mé API!"}
- Automatická dokumentace na http://127.0.0.1:8000/docs

**Vysvětlení:** 
Tento kód vytvoří základní FastAPI aplikaci s jedním endpointem. `FastAPI()` vytvoří instanci aplikace, kterou můžeme použít k definování endpointů pomocí dekorátorů jako `@app.get()`. Endpoint `/` vrací jednoduchý JSON objekt.

---

# 2. Path a Query parametry
## Co demonstruje:
Jak pracovat s cestami (path) a dotazními parametry (query) v FastAPI.

```python
from fastapi import FastAPI, Path, Query

app = FastAPI()

# Endpoint s path parametrem
@app.get("/uzivatele/{user_id}")
async def get_user(user_id: int = Path(..., ge=1, le=1000)):
    return {"user_id": user_id, "message": f"Nalezen uživatel {user_id}"}

# Endpoint s query parametrem
@app.get("/produkty")
async def get_products(name: str = Query(None), min_price: float = Query(None)):
    response = {"message": "Seznam produktů"}
    if name:
        response["name"] = name
    if min_price is not None:
        response["min_price"] = min_price
    return response

# Spuštění příkazem: uvicorn app:app --reload
```

**Očekávaný výstup:**
- `GET /uzivatele/42` → {"user_id": 42, "message": "Nalezen uživatel 42"}
- `GET /produkty?name=notebook&min_price=5000` → {"message": "Seznam produktů", "name": "notebook", "min_price": 5000.0}

**Vysvětlení:**
- `Path(..., ge=1, le=1000)` zajišťuje, že parametr musí být celé číslo mezi 1 a 1000
- `Query(None)` umožňuje volitelný query parametr s výchozí hodnotou None
- FastAPI automaticky validuje vstupy podle typů a pravidel

---

# 3. Request/Response modely (Pydantic)
## Co demonstruje:
Použití Pydantic modelů pro definici struktur požadavků a odpovědí.

```python
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional

app = FastAPI()

# Definice modelu pro uživatele (response)
class UserResponse(BaseModel):
    id: int
    name: str
    email: str
    is_active: bool = True

# Definice modelu pro vytvoření uživatele (request)
class UserCreate(BaseModel):
    name: str
    email: str
    password: str

# Seznam uživatelů (simulace databáze)
users_db = []

@app.post("/uzivatele", response_model=UserResponse)
async def create_user(user: UserCreate):
    # Simulace vytvoření uživatele
    new_user = {
        "id": len(users_db) + 1,
        "name": user.name,
        "email": user.email,
        "is_active": True
    }
    users_db.append(new_user)
    return new_user

# Spuštění příkazem: uvicorn app:app --reload
```

**Očekávaný výstup:**
- `POST /uzivatele` s JSONem:
```json
{
  "name": "Jan Novák",
  "email": "jan@example.com",
  "password": "heslo123"
}
```
- Odpověď:
```json
{
  "id": 1,
  "name": "Jan Novák",
  "email": "jan@example.com",
  "is_active": true
}
```

**Vysvětlení:**
- `BaseModel` z Pydanticu definuje strukturu dat
- `UserCreate` je model pro přijímání dat (request)
- `UserResponse` je model pro odesílání dat (response)
- FastAPI automaticky serializuje a validuje data podle modelů

---

# 4. Validace dat a middleware
## Co demonstruje:
Validaci dat, použití middleware a základní zabezpečení.

```python
from fastapi import FastAPI, HTTPException, Request
from pydantic import BaseModel, validator
from typing import List
import time

app = FastAPI()

# Model s validací
class Product(BaseModel):
    name: str
    price: float
    category: str
    
    @validator('price')
    def price_must_be_positive(cls, v):
        if v < 0:
            raise ValueError('Cena musí být kladná')
        return v

# Middleware pro logování requestů
@app.middleware("http")
async def log_requests(request: Request, call_next):
    start_time = time.time()
    response = await call_next(request)
    process_time = time.time() - start_time
    print(f"Zpracování trvalo {process_time:.2f} sekund")
    return response

# Endpoint pro vytvoření produktu s validací
@app.post("/produkt")
async def create_product(product: Product):
    return {"message": "Produkt vytvořen", "product": product}

# Spuštění příkazem: uvicorn app:app --reload
```

**Očekávaný výstup:**
- `POST /produkt` s JSONem:
```json
{
  "name": "Notebook",
  "price": 15000,
  "category": "Elektronika"
}
```
- Odpověď:
```json
{
  "message": "Produkt vytvořen",
  "product": {
    "name": "Notebook",
    "price": 15000.0,
    "category": "Elektronika"
  }
}
```

**Vysvětlení:**
- `@validator` vytváří vlastní validaci dat (cena nemůže být záporná)
- Middleware zachycuje každý request a loguje čas zpracování
- FastAPI automaticky kontroluje typy a validuje data podle Pydantic modelů

---

# 5. Autentizace a autorizace
## Co demonstruje:
Základní autentizaci pomocí API klíčů a základní autorizaci.

```python
from fastapi import FastAPI, HTTPException, Depends, Header
from pydantic import BaseModel
from typing import Optional

app = FastAPI()

# Simulace databáze uživatelů
USERS_DB = {
    "admin_key": {"username": "admin", "role": "admin"},
    "user_key": {"username": "user", "role": "user"}
}

# Model pro přihlášení
class LoginRequest(BaseModel):
    username: str
    password: str

# Závislost pro ověření API klíče
def verify_api_key(api_key: str = Header(...)):
    if api_key not in USERS_DB:
        raise HTTPException(status_code=401, detail="Neplatný API klíč")
    return USERS_DB[api_key]

# Závislost pro admin práva
def require_admin(user_data: dict = Depends(verify_api_key)):
    if user_data["role"] != "admin":
        raise HTTPException(status_code=403, detail="Přístup zamítnut")
    return user_data

# Endpoint bez autentizace
@app.get("/")
async def root():
    return {"message": "Veřejná stránka"}

# Endpoint vyžadující autentizaci
@app.get("/uzivatele", dependencies=[Depends(verify_api_key)])
async def get_users():
    return [{"id": 1, "name": "Jan Novák"}, {"id": 2, "name": "Petr Svoboda"}]

# Endpoint vyžadující admin práva
@app.delete("/uzivatele/{user_id}", dependencies=[Depends(require_admin)])
async def delete_user(user_id: int):
    return {"message": f"Uživatel {user_id} smazán"}

# Spuštění příkazem: uvicorn app:app --reload
```

**Očekávaný výstup:**
- `GET /uzivatele` s hlavičkou `api_key: user_key` → seznam uživatelů
- `DELETE /uzivatele/1` s hlavičkou `api_key: admin_key` → {"message": "Uživatel 1 smazán"}
- `DELETE /uzivatele/1` s hlavičkou `api_key: user_key` → HTTP 403 chyba

**Vysvětlení:**
- `verify_api_key()` je závislost, která ověřuje API klíč a vrací informace o uživateli
- `require_admin()` je další závislost, která kontroluje, že uživatel má admin práva
- `Depends()` umožňuje použít závislosti pro ověření a autorizaci
- FastAPI automaticky zpracovává HTTP hlavičky a validuje přihlášení

---

# Instalace potřebných balíčků:
```bash
pip install fastapi uvicorn pydantic
```

# Spuštění aplikací:
```bash
uvicorn app:app --reload
```

Všechny příklady jsou spustitelné samostatn

## 🎯 Cvičení a úkoly

<div style="background: #fff3e0; padding: 20px; border-left: 5px solid #ff9800; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">✍️ Praktická cvičení k procvičení</h3>
    <p>Vyřešte následující úkoly. Začněte od jednoduššších a postupujte k složitějším.</p>
</div>

# Cvičení pro kapitolu: REST API vytvoření s FastAPI

---

## Cvičení 1: Základní CRUD operace s uživateli

### **Název úkolu**
Vytvoření základního REST API pro správu uživatelů s CRUD operacemi.

### **Detailní zadání**
Vytvoř REST API pomocí FastAPI, které umožňuje vytvářet, číst, aktualizovat a mazat uživatele. Uživatel má následující atributy: id (integer), jméno (string), e-mail (string) a věk (integer).

### **Vstupní data/požadavky**
- POST `/users/` s JSON daty:
  ```json
  {
    "name": "Jan Novák",
    "email": "jan.novak@example.com",
    "age": 30
  }
  ```
- GET `/users/{user_id}`
- PUT `/users/{user_id}` s aktualizovanými daty
- DELETE `/users/{user_id}`

### **Očekávaný výstup**
- POST: Vrátí nově vytvořeného uživatele s přiřazeným ID.
- GET: Vrátí informace o konkrétním uživateli.
- PUT: Aktualizuje uživatele a vrátí aktualizované data.
- DELETE: Potvrdí smazání uživatele.

### **Hints/Nápověda**
1. Použij `pydantic` model pro definici datové struktury uživatele.
2. Vytvoř seznam (např. `users_db`) jako vnitřní paměť pro ukládání uživatelů.
3. Použij `@app.get`, `@app.post`, `@app.put`, `@app.delete` na definici endpointů.
4. Implementuj validaci a zpracování chyb při neexistujícím ID.

### **Kostra řešení**
```python
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Optional

app = FastAPI()

class User(BaseModel):
    id: int
    name: str
    email: str
    age: int

users_db = []

@app.post("/users/")
def create_user(user: User):
    users_db.append(user)
    return user

# Implementuj další endpointy (GET, PUT, DELETE)

```

### **Bonusové rozšíření**
- Přidání validace e-mailu pomocí regulárních výrazů.
- Přidání možnosti hledání uživatele podle jména.

---

## Cvičení 2: REST API pro knihovní systém

### **Název úkolu**
Vytvoření API pro správu knih v knihovně s možností hledání a filtrování.

### **Detailní zadání**
Vytvoř FastAPI aplikaci pro správu knih, která podporuje vytváření, čtení, aktualizaci a mazání knih. Přidej možnost vyhledávání podle názvu nebo autora.

### **Vstupní data/požadavky**
- POST `/books/` s JSON daty:
  ```json
  {
    "title": "1984",
    "author": "George Orwell",
    "year": 1948,
    "genre": "Dystopie"
  }
  ```
- GET `/books/` (všechny knihy)
- GET `/books/?author=George Orwell`
- PUT `/books/{book_id}` a DELETE `/books/{book_id}`

### **Očekávaný výstup**
- Všechny operace mají být správně ovládané s chybovými hláškami.
- GET s parametrem `author` filtruje knihy podle autora.

### **Hints/Nápověda**
1. Použij `Optional` pro volitelné parametry v URL.
2. Implementuj filtrování pomocí `query parameters`.
3. Využij `if` podmínky pro zjištění, zda kniha existuje.
4. Použij `HTTPException` pro chyby.

### **Kostra řešení**
```python
from fastapi import FastAPI, HTTPException

app = FastAPI()

books_db = []

@app.get("/books/")
def get_books(author: Optional[str] = None):
    if author:
        return [book for book in books_db if book.author == author]
    return books_db

# Implementuj POST, PUT, DELETE endpointy
```

### **Bonusové rozšíření**
- Přidání možnosti vyhledávání podle roku nebo žánru.
- Přidání stránkování výsledků.

---

## Cvičení 3: REST API s autentizací a autorizací

### **Název úkolu**
Vytvoření API s podporou přihlášení a zabezpečených endpointů.

### **Detailní zadání**
Vytvoř FastAPI aplikaci, která umožňuje uživatelům se přihlásit pomocí jména a hesla. Po přihlášení poskytne token, který je potřeba pro přístup k chráněným endpointům.

### **Vstupní data/požadavky**
- POST `/login` s:
  ```json
  {
    "username": "admin",
    "password": "secret"
  }
  ```
- GET `/protected` (po přihlášení)

### **Očekávaný výstup**
- Po úspěšném přihlášení dostane uživatel JWT token.
- Chráněné endpointy vyžadují token v hlavičce `Authorization`.

### **Hints/Nápověda**
1. Použij `python-jose` pro práci s JWT tokeny.
2. Vytvoř funkci pro ověření tokenu.
3. Implementuj middleware pro kontrolu autorizace.
4. Použij `HTTPException` s kódem 401 při neúspěšném přihlášení.

### **Kostra řešení**
```python
from fastapi import FastAPI, Depends, HTTPException
from jose import JWTError, jwt
from datetime import datetime, timedelta

app = FastAPI()

SECRET_KEY = "your-secret-key"
ALGORITHM = "HS256"

@app.post("/login")
def login(username: str, password: str):
    if username == "admin" and password == "secret":
        # Generuj JWT token
        return {"token": token}
    raise HTTPException(status_code=401, detail="Invalid credentials")

@app.get("/protected")
def protected_route(token: str = Depends(get_current_user)):
    return {"message": "This is a protected route"}
```

### **Bonusové rozšíření**
- Implementuj vypršení tokenu.
- Přidání možnosti registrace nových uživatelů.

---

## Cvičení 4: REST API s omezením počtu požadavků (Rate Limiting)

### **Název úkolu**
Přidání omezení počtu požadavků na endpointy.

### **Detailní zadání**
Vytvoř FastAPI aplikaci, která má všechny endpointy omezené na 5 požadavků za minutu pro každého uživatele (pomocí IP adresy).

### **Vstupní data/požadavky**
- GET `/api/data`
- Odeslání více požadavků v krátké době.

### **Očekávaný výstup**
- Prvních 5 požadavků proběhne bez problému.
- Při překročení limitu vrátí HTTP status 429 (Too Many Requests).

### **Hints/Nápověda**
1. Použij `fastapi-limiter` nebo vlastní mechanismus počítání požadavků.
2. Ulož počty požadavků do slovníku podle IP adresy.
3. Implementuj časový reset limitu (např. ka


## 📚 Další zdroje a materiály

<div style="background: #e3f2fd; padding: 20px; border-left: 5px solid #2196f3; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🔗 Doporučené materiály k dalšímu studiu</h3>
</div>

Jistě! Níže najdeš komplexní seznam zdrojů pro kapitolu **REST API vytvoření s FastAPI**, který můžeš použít pro výuku, studium nebo vlastní vývoj.

---

## 📚 Zdroje pro REST API vytvoření s FastAPI

### ✅ 5 doporučených článků / tutoriálů

1. **[FastAPI Official Documentation](https://fastapi.tiangolo.com/)**  
   - *Popis:* Oficiální dokumentace FastAPI. Obsahuje detailní vysvětlení všech funkcí, od základů až po pokročilé techniky jako např. dependency injection, validation, security.
   
2. **[Real Python: Build a REST API with Python and FastAPI](https://realpython.com/python-fastapi/)**  
   - *Popis:* Praktický výukový článek pro začátečníky, který vysvětluje základy FastAPI a jak vytvořit REST API s použitím SQLAlchemy a Pydantic.

3. **[Medium: Building a REST API with FastAPI and PostgreSQL](https://medium.com/@joshuakarjala/building-a-rest-api-with-fastapi-and-postgresql-650e79b4d118)**  
   - *Popis:* Článek ukazuje, jak vytvořit REST API s databází PostgreSQL a použít FastAPI pro CRUD operace.

4. **[FreeCodeCamp: FastAPI Tutorial for Beginners](https://www.freecodecamp.org/news/fastapi-tutorial/)**  
   - *Popis:* Úvodní tutorial určený pro začátečníky, který vysvětluje základní koncepty a příklady kódu pro tvorbu REST API.

5. **[Dev.to: Complete FastAPI Tutorial with Authentication and Database Integration](https://dev.to/andrewroberts/complete-fastapi-tutorial-with-authentication-and-database-integration-4k2p)**  
   - *Popis:* Komplexní průvodce, který zahrnuje autentizaci, databázi a správu tokenů.

---

### 🎥 3 YouTube videa

1. **[FastAPI Tutorial for Beginners - Python API Development](https://www.youtube.com/watch?v=7t2alSn_7Go)**  
   - *Délka:* 45 min  
   - *Popis:* Úvodní video pro začátečníky, které vysvětluje základy a ukazuje tvorbu jednoduchého API s použitím FastAPI.

2. **[FastAPI REST API with PostgreSQL and SQLModel](https://www.youtube.com/watch?v=8Fb9j0q7VYI)**  
   - *Délka:* 1 hodina  
   - *Popis:* Video ukazuje, jak vytvořit REST API s databází PostgreSQL a použít SQLModel pro modelování dat.

3. **[FastAPI Authentication and JWT Tutorial](https://www.youtube.com/watch?v=80k60hD9j2o)**  
   - *Délka:* 50 min  
   - *Popis:* Tento kurz se zaměřuje na autentizaci pomocí JWT tokenů a jejich použití ve FastAPI aplikacích.

---

### 📘 3 knihy / oficiální dokumentace

1. **[FastAPI Documentation](https://fastapi.tiangolo.com/)**  
   - *Popis:* Oficiální dokumentace – výčet funkcí, příklady, pokyny pro použití, doporučené postupy.

2. **[Python Web Development with FastAPI by Michael Herman](https://www.packtpub.com/product/python-web-development-with-fastapi/9781803241496)**  
   - *Popis:* Knížka se zaměřuje na tvorbu moderních API pomocí FastAPI, včetně autentizace, databází a deploymentu.

3. **[FastAPI by Example - Building a Full Stack Web App with Python and FastAPI](https://www.packtpub


## 📝 Shrnutí kapitoly

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white; margin: 30px 0;">
    <h3 style="color: white; margin-top: 0;">✅ Co jste se naučili</h3>
    <ul style="list-style: none; padding-left: 0;">
        <li>✓ FastAPI installation a setup</li>
<li>✓ Path a query parameters</li>
<li>✓ Request/Response models (Pydantic)</li>
<li>✓ Data validation</li>
<li>✓ Middleware a dependencies</li>
    </ul>
    
    <h3 style="color: white;">🎯 Klíčové dovednosti</h3>
    <p>Po dokončení této kapitoly byste měli být schopni prakticky použít všechny probrané koncepty.</p>
    
    <h3 style='color: white;'>➡️ Další kapitola</h3><p>Kapitola 19 - pokračujte ve studiu!</p>
</div>

---

*📅 Notebook vygenerován: 2025-09-29 12:31:37*  
*🤖 Generátor: Comprehensive Colab Generator v2.0*  
*📚 Učebnice programování - Od základů k AI*


## 🧪 Sandbox - Prostor pro experimenty

Použijte následující buňky pro vlastní experimenty a testování:

In [ ]:
# 🧪 Zde můžete experimentovat s kódem z kapitoly
# Napište svůj kód zde:

